In [ ]:
!pip install mediapipe==0.10.20 opencv-python-headless


In [ ]:
#IMPORT REQUIRED LIBRARIES
import cv2 # computer vision for image reading ,Processing and Transforming and to calculate EAR
import mediapipe as mp #Mediapipe for facial frecognition for facial points to compute EAR(eye aspect ratio)
import numpy as np # Numpy for image tensor computations
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


In [ ]:
!wget -q https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/latest/face_landmarker.task
#importing pretrained facial points detector (facelandmarker) - Transfer lear

In [ ]:
base_options = python.BaseOptions(model_asset_path="face_landmarker.task")

options = vision.FaceLandmarkerOptions(
    base_options=base_options, #Loading the imported model
    running_mode=vision.RunningMode.VIDEO,
    num_faces=1 # Limit on how many faces to detec
)

detector = vision.FaceLandmarker.create_from_options(options)


In [ ]:
#EAR = (distance between upper and lower eyelids / width of eye lids)
def compute_EAR(landmarks, eye_indices):
    p = [landmarks[i] for i in eye_indices]

    def dist(a, b):
        return np.linalg.norm(np.array([a.x, a.y]) - np.array([b.x, b.y]))

    vertical1 = dist(p[1], p[5])
    vertical2 = dist(p[2], p[4])
    horizontal = dist(p[0], p[3])

    return (vertical1 + vertical2) / (2.0 * horizontal)


In [ ]:
#Video input and output paths
input_video = "/content/input.mp4"
output_video = "output.mp4"
#Reading video using cv2
cap = cv2.VideoCapture(input_video)

fps = cap.get(cv2.CAP_PROP_FPS) #calculates Fps of an video
w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))#Width of frame of the video
h  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))#height of frame of the video

out = cv2.VideoWriter(output_video,
                      cv2.VideoWriter_fourcc(*'mp4v'),
                      fps, (w, h))

LEFT_EYE  = [33,160,158,133,153,144] #Landmarks of left eye
RIGHT_EYE = [362,385,387,263,373,380]#Landmarks of right eye

frame_timestamp = 0
drowsy_counter = 0 #Counter to store the count of number of frames the person closed his eyes
EAR_THRESHOLD = 0.23 #Threshold for checking if the eye is closed or not , below 0.23 is closed
FRAMES_THRESHOLD = 50 #Number of frames to check if person is drowsy or not


In [ ]:
while cap.isOpened(): #iterate through frames
    ret, frame = cap.read() #ret returns True if frame exist , frame reads the numpy tensor of image in BGR format
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)#Converting BGR to RGB

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb) #Transform the numpy image tensor into the required format of mediapipe

    result = detector.detect_for_video(mp_image, frame_timestamp) # this detects the face and returns 436 landmarks of face
    frame_timestamp += int(1000 / fps)

    status = "Awake"

    if result.face_landmarks:
        landmarks = result.face_landmarks[0]

        left_EAR  = compute_EAR(landmarks, LEFT_EYE) #Computing EAR for left eye
        right_EAR = compute_EAR(landmarks, RIGHT_EYE)#computing EAR for right eye

        EAR = (left_EAR + right_EAR) / 2.0 # Average of both to see if he closed both are not

        if EAR < EAR_THRESHOLD: #If less EAR less than threshold , drowsy counter increases Else again set to '0'
            drowsy_counter += 1
        else:
            drowsy_counter = 0

        if drowsy_counter > FRAMES_THRESHOLD:#If eyes were closed more than 50 frames then set status to 'DROWSY'
            status = "DROWSY!"

        cv2.putText(frame, f"EAR:{EAR:.3f}", (30,40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2) #Overlay the message of the score of EAR

    color = (0,255,0) if status=="Awake" else (0,0,255) #Overlay the message if the person is awake or not

    cv2.putText(frame, status, (30,80),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3) #Add the overlays to the current frame

    out.write(frame) #Write each frame to the video

cap.release()
out.release()


In [ ]:
from google.colab import files
files.download("output.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>